读取数据。

In [ ]:
import torch # pytorch 主库
from torch import nn # 神经网络
from torch.nn import functional as F # 激活函数
from d2l import torch as d2l # 本书自定义的库
import random # 随机数
import pandas as pd # 读取数据
import numpy as np # 数组
import math


In [ ]:
batch_size, num_steps = 32, 35

## 我们首先要做的是处理数据，第一步是读取数据


In [ ]:
time_series = pd.read_csv('/home/ubuntu/code/EAST/data/2024-09-20/POINT_N/shot_77003_data.csv')
# 此时我们的 point_ne 格式为 (4250000, 12)，行是样本量，列是特征量
time_series.shape

In [ ]:
time_series.min()

## 接下来我们要定义抽样方法，设计迭代器

In [ ]:
# 顺序抽样
def seq_data_iter_sequential(time_series, batch_size, num_steps):
    """使用顺序分区生成一个小批量子序列"""
    """ 输入语料库，批量大小，时间步数，返回小批量子序列"""
    
    offset = random.randint(0, num_steps)
    # 从随机偏移量开始划分序列
    num_tokens = ((len(time_series) - offset - 1) // batch_size) * batch_size
    # 计算有多少个有效的样本
    # 为什么要这么计算呢？
    ## 首先是搞了个偏移量，让我们的结果有随机性，生成的小批量子序列不同
    ## 其次我们最终选到的数据实际上是 batch_size 的整数倍，因此 num_tokens 是有效样本量
    Xs = torch.tensor(time_series[offset: offset + num_tokens, :])
    Ys = torch.tensor(time_series[offset + 1: offset + 1 + num_tokens, :])
    # 计算有效的输入和标签
    # 我们的 time_series 是 (样本量, 特征) 的格式
    Xs, Ys = Xs.reshape(batch_size, -1, 11), Ys.reshape(batch_size, -1, 11)
    # 将输入和标签转换为 (小样本量, -1, 特征数) 的格式
    num_batches = Xs.shape[1] // num_steps
    # 计算有多少个批量
    for i in range(0, num_steps * num_batches, num_steps):
        X = Xs[:, i: i + num_steps, :]
        Y = Ys[:, i: i + num_steps, :]
        yield X, Y

In [ ]:
# 说明咱们的迭代是可行的
for X, Y in seq_data_iter_sequential(point_ne.values, batch_size, num_steps):
    print(X.shape)

In [ ]:
def seq_data_iter_random(time_series, batch_size, num_steps):
    """使用随机抽样生成一个小批量子序列"""
    
    time_series = time_series[random.randint(0, num_steps - 1):]
    # 从随机偏移量开始对序列进行分区，随机范围为[0, num_steps - 1]
    # 减去1，是因为我们需要考虑标签
    # random.randint(a, b) 返回一个随机整数N，a <= N <= b

    num_subseqs = (len(time_series) - 1) // num_steps
    # 一个子序列的长度为num_steps，所以有len(corpus) - 1 // num_steps 个子序列
    # // 的意思是整除
    # 减去1，是因为我们需要考虑标签

    initial_indices = list(range(0, num_subseqs * num_steps, num_steps))
    # 生成一个包含所有子序列的起始索引的列表
    # range(start, end, step) 返回一个包含等差数列的列表

    random.shuffle(initial_indices)
    # 将所有子序列的起始索引打乱
    # random.shuffle() 方法将序列的所有元素随机排序
    # 在随机抽样的迭代过程中，
    # 来自两个相邻的、随机的、小批量中的子序列不一定在原始序列上相邻

    def data(pos):
        return time_series[pos: pos + num_steps, :]
        # 返回从pos位置开始的长度为num_steps的序列

    num_batches = num_subseqs // batch_size
    # 一个批量的子序列的数量为num_subseqs // batch_size

    for i in range(0, batch_size * num_batches, batch_size):
        
        initial_indices_per_batch = initial_indices[i: i + batch_size]
        # 在这里，initial_indices包含子序列的随机起始索引
        # initial_indices_per_batch包含了一个批量的子序列的随机起始索引
        
        X = [data(j) for j in initial_indices_per_batch]
        Y = [data(j + 1) for j in initial_indices_per_batch]
        yield torch.tensor(X), torch.tensor(Y)
        # X, Y 的格式是 (batch_size, num_steps, 特征数)


In [ ]:
# 说明咱们的迭代是可行的
time_series = point_ne.iloc[:10000]
for X, Y in seq_data_iter_random(time_series.values, batch_size, num_steps):
    print(X.shape)

## 将两个抽样函数打包

In [ ]:
class SeqDataLoader:
    """加载序列数据的迭代器"""
    # 输入 batch_size, num_steps, use_random_iter, max_tokens，返回按顺序或者随机的方式小样本抽样的一个迭代器
    def __init__(self, batch_size, num_steps, use_random_iter, max_tokens):
        # max_tokens是指读取 time_series 的最大数量，截取前max_tokens个，目前弃用状态
        if use_random_iter:
            self.data_iter_fn = seq_data_iter_random
        # 如果use_random_iter为True，则使用随机抽样
        else:
            self.data_iter_fn = seq_data_iter_sequential
        # 否则使用顺序抽样
        self.time_series = load_data_timeseries(max_tokens)
        # 读取时光机器数据集
        # 输入的max_tokens是指读取 time_series 的最大数量
        self.batch_size, self.num_steps = batch_size, num_steps

    def __iter__(self):
        return self.data_iter_fn(self.time_series, self.batch_size, self.num_steps)
    # 返回迭代器

def load_data_timeseries(max_tokens=-1):
    """ 返回时间序列数据集 """
    time_series = pd.read_csv('/home/ubuntu/code/EAST/data/2024-09-20/POINT_N/shot_77003_data.csv')

    if max_tokens > 0:
        time_series = time_series[:max_tokens]
    return time_series.iloc[:,1:].values
    # 如果 max_tokens 大于 0，则返回前 max_tokens 个词元索引，否则返回所有索引

## 最后我们定义一个函数，直接返回数据迭代器

In [ ]:
# 定义一个函数，返回迭代器
# 至于我们要加载哪些数据，已经内置在 SeqDataLoader 类中，更进一步load_data_timeseries 函数中
def load_data_east(batch_size, num_steps, 
                           use_random_iter=False, max_tokens=10000):
    # use_random_iter是指是否使用随机抽样
    # max_tokens是指将corpus中的词转换为标记的最大数量，截取前max_tokens个词
    
    """返回时光机器数据集的迭代器和词表"""
    data_iter = SeqDataLoader(
        batch_size, num_steps, use_random_iter, max_tokens)
    # 创建一个SeqDataLoader对象
    # 输入 batch_size, num_steps, use_random_iter, max_tokens，返回按顺序或者随机的方式小样本抽样的一个迭代器
    return data_iter

# 循环神经网络的从零开始实现

In [ ]:
# 初始化参数
def get_params(features, labels, num_hiddens, device):
    # num_hiddens 超参数，隐藏单元的数量
    num_inputs = features
    num_outputs = labels
    # 输入层参数等于输出层参数等于词表大小

    def normal(shape):
        return torch.randn(size=shape, device=device, dtype=torch.float64) * 0.01
    # 定义一个辅助函数，用来初始化模型参数
    # 根据输入的形状shape返回一个张量，张量的值是从标准正态分布中随机取样的，然后乘以0.01
    # device是用来指定张量的存储设备的，如果device是一个cuda设备，那么返回的张量将会被存在显存上

    # 隐藏层参数
    W_xh = normal((num_inputs, num_hiddens))
    W_hh = normal((num_hiddens, num_hiddens))
    b_h = torch.zeros(num_hiddens, device=device, dtype=torch.float64)
    # 输出层参数
    W_hq = normal((num_hiddens, num_outputs))
    b_q = torch.zeros(num_outputs, device=device, dtype=torch.float64)
    """ 附加梯度 """
    params = [W_xh, W_hh, b_h, W_hq, b_q]
    for param in params:
        param.requires_grad_(True)
    return params
    # 为模型参数附加梯度
    # 附加梯度是指在反向传播过程中，计算梯度的张量


In [ ]:
# 初始化隐藏状态
def init_rnn_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device, dtype=torch.float64), )
# 初始化隐藏状态，返回一个元组，元组中的第一个元素是一个张量，张量的形状是 (batch_size, num_hiddens)


In [ ]:
def rnn(inputs, state, params):
    # inputs的形状：(时间步数量，批量大小，特征数)
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    # H, 表示隐藏状态
    outputs = []
    # X的形状：(批量大小，特征数)
    for X in inputs:
        H = torch.tanh(torch.mm(X, W_xh) + torch.mm(H, W_hh) + b_h)
        # 计算隐藏状态
        Y = torch.mm(H, W_hq) + b_q
        # 计算输出
        outputs.append(Y)
    return torch.cat(outputs, dim=0), (H,)
    # 返回输出和隐藏状态
    # torch.cat()函数是将一个张量列表拼接成一个张量

In [ ]:
class RNNModelScratch: #@save
    """从零开始实现的循环神经网络模型"""
    def __init__(self, features, labels, num_hiddens, device,
                 get_params, init_state, forward_fn):
        self.features, self.labels, self.num_hiddens = features, labels, num_hiddens
        self.params = get_params(features, labels, num_hiddens, device)
        self.init_state, self.forward_fn = init_state, forward_fn

    def __call__(self, X, state):
        X = X.transpose(0, 1)
        return self.forward_fn(X, state, self.params)
    # __call__函数是一个特殊函数，当实例被当做函数调用时，会调用这个函数
    # X 将输入转换成 one-hot 向量，大小为 (时间步数, 批量大小, 词表大小)
    # 调用forward_fn函数，返回输出和隐藏状态，参数
    # 这里 forward_fn 函数是 rnn 函数

    def begin_state(self, batch_size, device):
        return self.init_state(batch_size, self.num_hiddens, device)
    # 返回初始化的隐藏状态

In [ ]:
# 检查输出是否正确

num_hiddens = 512 # 设置隐藏单元数量
features = 11 # 特征数
labels = 11 # 输出数
net = RNNModelScratch(features, labels, num_hiddens, d2l.try_gpu(), get_params,
                      init_rnn_state, rnn)
# 实例化 RNNModelScratch 类，输入词表大小 28，隐藏层大小 512，设备，初始化参数函数，初始化隐藏状态函数，循环神经网络函数
X = torch.zeros((batch_size, num_steps, features), dtype=torch.float64)
state = net.begin_state(batch_size, d2l.try_gpu())
# 初始化隐藏状态，全是 0
# 返回一个元组，元组中的第一个元素是一个张量，张量的形状是 (batch_size, num_hiddens)，第二个元素是空
# X 是一个 2 行 5 列的矩阵，它的行九室批量大小，列是时间步数
Y, new_state = net(X.to(d2l.try_gpu()), state)
# X 转化成所需的格式，时间步数 x 批量大小 x 词表大小
# 再输入当前隐藏层，进入 rnn 计算得到输出
Y.shape, len(new_state), new_state[0].shape

In [ ]:
def predict_ch8(prefix, num_preds, net, device):
    """ 预测 prefix 之后的数值 """
    # num_steps 表示预测的步数
    state = net.begin_state(batch_size=1, device=device)
    H, = state
    # 初始化隐藏状态，只需输入批量大小，得到 state，格式为 H,
    outputs = [prefix[0,:]]
    # 输出列表
    features = 11
    get_input = lambda: torch.tensor([outputs[-1]], device=device).reshape((1, 1, features))
    # 定义一个函数，返回一个张量，张量的值是输出列表的最后一个元素，形状是 (features, )
    for y in prefix[1:,:]:  # 预热期
        _, state = net(get_input(), state)
        # 输入当前数值和隐藏层状态，得到输出和新的隐藏层状态
        outputs.append(y)
        # 输出列表添加当前字符
    for _ in range(num_preds):  # 预测num_preds步
        y, state = net(get_input(), state)
        # 输入当前字符和隐藏层状态，得到输出和新的隐藏层状态
        outputs.append(list(y.T))
    # outputs 转化为张量
    return torch.tensor(outputs, device=device)
    # outputs 的格式为 (num_preds + prefix.shape[0], features)

In [ ]:
num_preds = 20
test = point_ne[:,1:]
predict_ch8(test, num_preds, net, d2l.try_gpu())



In [ ]:
def grad_clipping(net, theta):
    """裁剪梯度"""
    if isinstance(net, nn.Module):
        params = [p for p in net.parameters() if p.requires_grad]
    else:
        params = net.params
    norm = torch.sqrt(sum(torch.sum((p.grad ** 2)) for p in params))
    # 计算梯度的 L2 范数
    if norm > theta:
        for param in params:
            param.grad[:] *= theta / norm

In [ ]:
# 训练

def train_epoch_ch8(net, train_iter, loss, updater, device, use_random_iter):
    """训练网络一个迭代周期（定义见第8章）"""
    state, timer = None, d2l.Timer()
    
    metric = d2l.Accumulator(2)
    # 训练损失，用预测值与标签的交叉熵损失之和
    for X, Y in train_iter:
    # 对于迭代的每一个结果
        if state is None or use_random_iter:
            # 在第一次迭代或使用随机抽样时初始化state
            state = net.begin_state(batch_size=train_iter.batch_size, device=device)
        else:
            if isinstance(net, nn.Module) and not isinstance(state, tuple):
                # state对于nn.GRU是个张量
                state.detach_()
            else:
                # state对于nn.LSTM或对于我们从零开始实现的模型是个张量
                for s in state:
                    s.detach_()
        y = Y.transpose(0, 1)
        y = Y.reshape(-1, Y.shape[2])
        X, y = X.to(device), y.to(device)
        y_hat, state = net(X, state)
        l = loss(y_hat, y.long())
        if isinstance(updater, torch.optim.Optimizer):
            updater.zero_grad()
            l.backward()
            grad_clipping(net, 1)
            updater.step()
        else:
            l.backward()
            grad_clipping(net, 1)
            # 因为已经调用了mean函数
            updater(batch_size=1)
        metric.add(l * y.numel(), y.numel())
    return math.exp(metric[0] / metric[1]), metric[1] / timer.stop()

In [ ]:
#@save
def train_ch8(net, train_iter, lr, num_epochs, device,
              use_random_iter=False):
    """训练模型（定义见第8章）"""
    loss = nn.MSELoss()
    animator = d2l.Animator(xlabel='epoch', ylabel='perplexity',
                            legend=['train'], xlim=[10, num_epochs])
    # 初始化
    if isinstance(net, nn.Module):
        updater = torch.optim.SGD(net.parameters(), lr)
    else:
        updater = lambda batch_size: d2l.sgd(net.params, lr, batch_size)
    
    time_series = load_data_timeseries(1000)
    predict = lambda prefix: predict_ch8(prefix, 50, net, device)
    
    
    # 训练和预测
    for epoch in range(num_epochs):
        ppl, speed = train_epoch_ch8(
            net, train_iter, loss, updater, device, use_random_iter)
        if (epoch + 1) % 10 == 0:
            
            #predict(time_series)
            animator.add(epoch + 1, [ppl])
    print(f'困惑度 {ppl:.1f}, {speed:.1f} 词元/秒 {str(device)}')
    #predict(time_series)

In [ ]:
num_epochs, lr = 500, 1
#train_iter = load_data_east(batch_size, num_steps, use_random_iter=True)
train_ch8(net, train_iter, lr, num_epochs, d2l.try_gpu())


In [ ]:
for X, Y in train_iter:
    print(X.shape, Y.shape)
    state = net.begin_state(batch_size=32, device=d2l.try_gpu())
    y_hat, state = net(X.to(d2l.try_gpu()), state)
    print(y_hat.shape)
    
    break

In [ ]:
load_data_timeseries().shape

In [ ]:
x = torch.tensor([1., 2, 3])
y = torch.tensor([4., 8, 6])
l = nn.MSELoss()
l(x, y)

## 简洁实现


In [ ]:
batch_size, num_steps = 32, 35 # 指定批量大小和时间步数
train_iter = load_data_east(batch_size, num_steps)
# load_data_time_machine 输入批量大小和时间步数，返回数据迭代器和词汇表

## [**定义模型**]

高级API提供了循环神经网络的实现。
我们构造一个具有256个隐藏单元的单隐藏层的循环神经网络层`rnn_layer`。
事实上，我们还没有讨论多层循环神经网络的意义（这将在 :numref:`sec_deep_rnn`中介绍）。
现在仅需要将多层理解为一层循环神经网络的输出被用作下一层循环神经网络的输入就足够了。

In [ ]:
num_hiddens = 256
features = 11
labels = 11
rnn_layer = nn.RNN(features, num_hiddens)

我们(**使用张量来初始化隐状态**)，它的形状是（隐藏层数，批量大小，隐藏单元数）。


In [ ]:
state = torch.zeros((1, batch_size, num_hiddens))
state.shape

In [ ]:
X = torch.rand(size=(num_steps, batch_size, features))
# torch.rand 函数生成一个形状为 (num_steps, batch_size, len(vocab)) 的小批量数据
Y, state_new = rnn_layer(X, state)
Y.shape, state_new.shape

In [ ]:
class RNNModel(nn.Module):
    """循环神经网络模型"""
    def __init__(self, rnn_layer, features, labels, **kwargs):
        super(RNNModel, self).__init__(**kwargs)
        self.rnn = rnn_layer
        self.features = features
        self.labels = labels
        self.num_hiddens = self.rnn.hidden_size
        # 如果RNN是双向的（之后将介绍），num_directions应该是2，否则应该是1
        if not self.rnn.bidirectional:
            self.num_directions = 1
            self.linear = nn.Linear(self.num_hiddens, self.features)
        else:
            self.num_directions = 2
            self.linear = nn.Linear(self.num_hiddens * 2, self.features)

    def forward(self, inputs, state):
        X = F.one_hot(inputs.T.long(), self.vocab_size)
        X = X.to(torch.float32)
        Y, state = self.rnn(X, state)
        # 全连接层首先将Y的形状改为(时间步数*批量大小,隐藏单元数)
        # 它的输出形状是(时间步数*批量大小,词表大小)。
        output = self.linear(Y.reshape((-1, Y.shape[-1])))
        return output, state

    def begin_state(self, device, batch_size=1):
        if not isinstance(self.rnn, nn.LSTM):
            # nn.GRU以张量作为隐状态
            return  torch.zeros((self.num_directions * self.rnn.num_layers,
                                 batch_size, self.num_hiddens),
                                device=device)
        else:
            # nn.LSTM以元组作为隐状态
            return (torch.zeros((
                self.num_directions * self.rnn.num_layers,
                batch_size, self.num_hiddens), device=device),
                    torch.zeros((
                        self.num_directions * self.rnn.num_layers,
                        batch_size, self.num_hiddens), device=device))